In [5]:
import pandas as pd
import json
import numpy as np    

# Load basic CI relational data
ci_df = pd.read_json("../sql/Configuration_Item.json")
attributes_df = pd.read_json("../sql/Attribute.json")

# Load performance relational data
rules_df = pd.read_json("../sql/perfRules.json")
rule_instances_df = pd.read_json("../sql/perfRuleInstances.json")
perf_data_df = pd.read_json("../sql/perfRuleData.json")

# Load monitors relational data
monitors_df = pd.read_json("../sql/Monitors.json")
monitor_instances_df = pd.read_json("../sql/MonitorInstances.json")
monitor_instance_states_df=pd.read_json("../sql/MonitorInstanceStates.json")
health_states_df=pd.read_json("../sql/HealthStates.json").set_index('state_id')


# Merge CI and attributes data into one dataframe
ci_att_df = pd.merge(ci_df,attributes_df,on="ci_id")
cp=ci_att_df.pivot(index="name", columns="att_name")
cp.columns = ['ci_id','ci_id2','ci_id3','device_type','env','ip_address']
cp.drop(columns=['ci_id2', 'ci_id3'], inplace=True)
configuration_items=cp.reset_index()
monitor_instances_df = pd.merge(monitor_instances_df,monitors_df,on="monitor_id")


# Merge performance dataframes
rule_instances_df.rename(columns={'per_ruleId':'ruleId'}, inplace=True)
rule_instances_df = pd.merge(ci_df,rule_instances_df,on="ci_id")
rule_instances = pd.merge(rule_instances_df,rules_df,on="ruleId")
rule_instances.sort_values('ci_id')






,ci_id,name,instance_id,ruleId,rule_name,rule_description
0,7,productivetrue.local,23,23,% Processor usage,% Processor usage
46,7,productivetrue.local,30,20,% Available Memory,% Available Memory
23,7,productivetrue.local,29,21,% Available Swap Space,% Available Swap Space
1,8,extraneousbent.local,7,23,% Processor usage,% Processor usage
47,8,extraneousbent.local,40,20,% Available Memory,% Available Memory
...,...,...,...,...,...,...
44,91,discreteexit.local,49,21,% Available Swap Space,% Available Swap Space
67,91,discreteexit.local,50,20,% Available Memory,% Available Memory
22,100,grouchybaker.local,24,23,% Processor usage,% Processor usage
45,100,grouchybaker.local,51,21,% Available Swap Space,% Available Swap Space


In [6]:
documents = []

for ci_index, configuration_item in configuration_items.iterrows():
    ci = json.loads(configuration_item.to_json())
    
    # Attach performance rules instances
    ci["performance_rules"]=[]
    for rule_index, rule_instance in rule_instances[rule_instances["ci_id"]==ci["ci_id"]].loc[:,'ruleId':'rule_description'].iterrows():
        rule = json.loads(rule_instance.to_json())
        
        # Attach performance data 
        data = []
        for perf_data_index, perf_data in perf_data_df[perf_data_df["perf_rule_instId"]==rule["ruleId"]].loc[:,'value':'date'].iterrows():
            d = json.loads(perf_data.to_json())
            data.append(d)
        rule["data"]=data
        ci["performance_rules"].append(rule)
        
    # Attach monitor instances
    ci["monitors"]= []
    
    for monitor_index, monitor_instance in monitor_instances_df[monitor_instances_df["ci_id"]==ci["ci_id"]].loc[:,['mon_isntance_id','name','description']].iterrows():
        monitor = json.loads(monitor_instance.to_json())
        
        # Attach health state for each monitor instance
        states = []
        
        for monitor_instance_index, monitor_instance_state in monitor_instance_states_df[monitor_instance_states_df["mon_isntance_id"]==monitor["mon_isntance_id"]].loc[:,['state_change_date','health_State_id']].iterrows():
            state_id=int(monitor_instance_state["health_State_id"])-1
            monitor_instance_state["health_State_id"] = health_states_df.iloc[state_id].state_name
            currentState = json.loads(monitor_instance_state.to_json())
            states.append(currentState)
        
        monitor["states"]=states
        
        ci["monitors"].append(monitor)
    
    documents.append(ci)
   
    
    



In [7]:
with open('../nosql/cis.json', 'w') as fout:
    fout.write(json.dumps(documents, indent=4))